# STABILIZING EFFECT OF VOLATILITY
## First Hitting Times of Market Returns for different Temporal Windows 
Questo codice esegue l'analisi di Stabilità del NASDAQ come proposta nel papero **Stabilizing effect of volatility in financial markets**.

Si calcolano i Tempi di Primo Passaggio medi dei Ritorni delle azioni in funzione della Volatilità di questi.

Si cerca di stabilire qual'è il numero minimo di giorni su cui analizzare il mercato per trovare un andamento **non-monotono** dei FHT.  

![First Hitting Times versus Volatility](./MFHT_fixdiff.png)

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime as dt
# Grafici interattivi sul browser con Plotly
import chart_studio.plotly as py
import plotly.graph_objects as go
# Imputer per dati mancanti
from sklearn.impute import KNNImputer

### Parametri iniziali

In [48]:
# Anno iniziale e finale. 
# I file di dati sono nominati per anni
START_DATE = '1996'
END_DATE = '2008'

# File sorgente coi dati
FILEPATH = f'../series/dprice_dp_{START_DATE}{END_DATE}.csv'
MAXSTOCK = 2000

# Moltiplicatori della stdev per la Soglia Alta e Bassa per la fuga
PARAM_START = -0.1
PARAM_CRASH = -1.5
# Limite superiore per i Ritorni (100%)
UPPER_LIMIT = 100

# Determina il FHT minimo e massimo per essere valido
TAU_MIN = 2
TAU_MAX = 300

# Determina la stdv massima e minima per essere valida
SIGMA_MAX = 1000
SIGMA_MIN = 0.01

# Numero di stock nel file.
# Serivirà a dare i nomi alle colonne del DataFrame.
NUMSTOCK = 617

# Lunghezza (in giorni) della prima finestra
STARTWIN = 31
# Numero di Finestre da generare a partire dalla prima
NUMWINS = 150

# Numero di canali in cui plottare il MFHT
NUMBINS = 50

# Si può controllare quanto output ricevere 
VERBOSE = False

## Estrazione dati
Qui si usa la libreria Pandas:
`import pandas as pd` 

Questa fornisce degli oggetti utili all'analisi dati: i **DataFrames** .

Sono come fogli di calcolo con righe e colonne intestate.

Il file sorgente è un file .csv contenente per colonne le Azioni del NASDAQ, 1665 in totale, 
e per righe i Ritorni Giornalieri, calcolati come aumento percentuale del Prezzo di Chiusura: 
$$\frac{p(i+1) - p(i)}{p(i)}$$
La prima colonna contiene le date dei giorni borsistici.

### Estrazione dei Ritorni e selezione
La funzione `polish_database(file)` estrae ed elabora un DataFrame da un file csv; per questo si usa anche la funzione `dateparse` per aggiustare il formato delle date e la funzione `find_good_stocks(df)` per selezionare le colonne del DataFrame con Ritorni in _formato numerico_ e con deviazioni standard nel range definito da `SIGMA_MIN` e `SIGMA_MAX`



In [4]:
def find_good_stocks(df):
    """
    Seleziona le azioni 'buone' secondo alcuni parametri:
    I Ritorni devono essere in formato numerico e la loro 
    Deviazione Standard deve essere compresa tra la massima 
    e la minima permesse (per prevenire stranezze).
    L'analisi procede colonna per colonna.
    Parameters
    ----------
    df : DataFrame
        DataFrame nelle cui colonne cercare le azioni 'buone'.
    Returns
    -------
    good_stocks
        Lista degl indici numerici delle azioni 'buone'
    """
    good_stocks = []
    bad_stocks = []
    for stock in df.columns:
        stock_series = df[stock]
        if stock_series.dtype != 'float64' or stock_series.std() < SIGMA_MIN or stock_series.std() > SIGMA_MAX:
            bad_stocks.append(stock)
        else:
            good_stocks.append(stock)
    print(f'{len(good_stocks)} good stocks remaining!')
    return good_stocks

def dateparse(date_string):
    """
    Analizza le date grezze di un file e le trasforma in formato data.
    Parameters
    ----------
    date_string : string
        Data in formato string
    Returns
    -------
    date
        Data in formato Datetime
    """
    date = dt.strptime(date_string, '%y%m%d')
    return date

def polish_database(file):
    """
    Estrae ed elabora il DataFrame. 
    Il DataFrame è estratto da un file csv di cui si indica il percorso.
    Le colonne sono nominate per numero.
    Le date nella prima colonna sono convertite in formato data
    dalla funzione dateparse.
    La colonna di date è quindi usata come indice.
    Parameters
    ----------
    file : string
        Percorso del file csv da cui estrarre i dati.
    Returns
    -------
    polished_df
        DataFrame elaborato
    """
    stock_numbers = list(range(0, NUMSTOCK))
    df = pd.read_csv(file,
                     header=0, names=stock_numbers,
                     parse_dates=[0], date_parser=dateparse, index_col=[0])

    # Gli stocks ben formattati sono di tipo float e non string.
    # Questi sono pochi meno di quelli originali.
    good_stocks = find_good_stocks(df)
    polished_df = df[good_stocks]
    return polished_df

In [5]:
# ESTRAZIONE DATI
database_stocks = polish_database(FILEPATH)
database_stocks

616 good stocks remaining!


,1,2,3,4,5,6,7,8,9,10,...,607,608,609,610,611,612,613,614,615,616
0,,,,,,,,,,,,,,,,,,,,,
1996-01-02,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1996-01-03,-0.031074,NaN,0.029412,0.021127,0.100000,0.063158,0.000000,0.073684,0.000000,-0.075000,...,-0.163934,-0.083333,0.037037,-0.055351,-0.382022,0.000000,0.411765,-0.090903,0.010101,-0.125000
1996-01-04,0.000000,NaN,0.057143,0.064909,-0.060606,0.365384,-0.098039,-0.196078,0.096418,-0.027027,...,-0.049218,0.000000,-0.004286,0.046875,-0.427273,0.038462,0.041667,0.149988,0.205000,0.428571
1996-01-05,-0.025641,NaN,0.108108,0.064935,0.000000,-0.140845,0.239130,0.097561,0.032259,0.083333,...,0.077721,-0.257576,0.009900,0.051571,-0.634921,-0.129630,0.120000,0.565246,0.136929,0.150000
1996-01-07,-0.178324,NaN,-0.060976,0.319495,0.064516,-0.016394,0.298246,0.266667,-0.039315,-0.025641,...,0.168908,0.020408,0.008928,-0.014285,-0.217391,-0.021276,0.428571,0.666652,0.459854,-0.043478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-24,0.000000,-0.028777,0.000000,0.018896,-0.007568,0.046286,0.000000,0.020234,0.072566,-0.015038,...,-0.005871,0.000000,0.030303,-0.007346,-0.046185,0.000000,-0.033113,0.020297,0.010381,-0.010753
2008-12-26,0.012820,0.029630,0.000000,-0.001854,0.028770,0.007492,0.000000,0.027460,0.021452,0.005089,...,-0.001969,0.000000,-0.014706,0.000000,-0.018947,0.000000,-0.002283,0.006532,0.003425,0.014650
2008-12-29,-0.022785,0.050360,-0.024000,-0.004459,-0.014488,-0.020184,0.012821,-0.063103,0.030695,-0.063291,...,-0.019724,0.096972,-0.004975,0.017576,0.043991,-0.037500,-0.020595,-0.021239,-0.033447,-0.005123


## Analisi preliminare dei dati
I dati buoni si trovano ora sistemati bene in un DataFrame.


### Deviazione Standard Media
Si ottiene la Deviazione Standard media dell'intero campione `total_stddev` come **media 
delle Deviazioni Standard** di tutte le azioni analizzate.

$$
    \sigma_{TOT} = \frac{\sum_{stocks}\sigma_{stock}}{N_{stocks}}
$$

*** 
***!!!*** _Questa Deviazione Standard Media è calcolata su un intervallo temporale di circa 10 anni, 
molto più grande di quello usato per ciascuna analisi di stabilità, che avviene in **finestre** di date dell'ordine di mesi._
_Si potrebbe rendere dinamico il calcolo della Deviazione Standard e quindi delle Soglie, finestra per finestra._

In [23]:
def print_output(dev, avg):
    """
    Stampa in StdOut le Medie e Deviazioni Standard
    dei primi cinque e degli ultimi cinque stocks.
    Stampa inoltre anche la Media totale delle Deviazioni Standard,
    la Massima e la Minima.
    Parameters
    ----------
    dev : Pandas Series 
        Deviazioni Standard dei Ritorni di ogni stock
    avg : Pandas Series
        Medie dei Ritorni di ciascuno stock
    Returns
    -------
    total_stddev : Pandas Series
        Deviazione Standard Media dell'intero set,
        cioè la media di tutte le Deviazioni.
    """
    for stock in dev.index[:5]:
        print(f'Stock {stock} \t Average = {avg[stock]:.4f} \t',
              f'StDev = {dev[stock]:.4f}')
    print('.\n.\n.')
    for stock in dev.index[-5:]:
        print(f'Stock {stock} \t Average = {avg[stock]:.4f} \t',
              f'StDev = {dev[stock]:.4f}')

    # Media delle deviazioni standard
    total_stddev = dev.sum()/len(dev)       

    print(f'\nMean Sigma: {total_stddev:.4}')
    print(f'Max Sigma: {dev.max():.4}')
    print(f'Min Sigma: {dev.min():.4}')
    return total_stddev

In [24]:
stddev = database_stocks.std(numeric_only=True)
# Si usa come indice l'elenco di stocks rimaste
stddev.index = database_stocks.columns
stddev

1      0.086495
2      0.361559
3      0.188615
4      0.095660
5      0.110985
         ...   
612    0.173995
613    0.367779
614    0.190163
615    0.154856
616    0.411635
Length: 616, dtype: float64

In [25]:
averages = database_stocks.mean(numeric_only=True)
# Si usa come indice l'elenco di stocks rimaste
averages.index = database_stocks.columns

In [26]:
# Media delle Deviazioni Standard di tutti gli stock analizzati
total_stddev = print_output(stddev, averages)

Stock 1 	 Average = 0.0034 	 StDev = 0.0865
Stock 2 	 Average = 0.0427 	 StDev = 0.3616
Stock 3 	 Average = 0.0132 	 StDev = 0.1886
Stock 4 	 Average = 0.0045 	 StDev = 0.0957
Stock 5 	 Average = 0.0069 	 StDev = 0.1110
.
.
.
Stock 612 	 Average = 0.0120 	 StDev = 0.1740
Stock 613 	 Average = 0.0405 	 StDev = 0.3678
Stock 614 	 Average = 0.0197 	 StDev = 0.1902
Stock 615 	 Average = 0.0127 	 StDev = 0.1549
Stock 616 	 Average = 0.0374 	 StDev = 0.4116

Mean Sigma: 0.2042
Max Sigma: 3.87
Min Sigma: 0.02826


### Calcolo delle Soglie
Le soglie Alta e Bassa e la soglia di Divergenza sono calcolate a partire dalla Deviazione Standard media, 
secondo i moltiplicatori definiti all'inizio.

Queste servono per cominciare e terminare il conteggio del FHT durante l'analisi.

![Calcolo di FHT](./conteggio_fht.png)

In [27]:
def calc_treshold(std):
    """
    Calcola le soglie Alta e Bassa per il calcolo di FHT e la soglia di Divergenza 
    per controllare comportamenti anomali dei Ritorni.
    Queste soglie sono proporzionali alla Deviazione Standard Media del set.
    Parameters
    ----------
    std : float
        Deviazione Standard Media del set
    Returns
    -------
    start : float
        Soglia Alta di partenza del conteggio di FHT
    divergenza : float
        Soglia limite per i Ritorni
    crash : float
        Soglia Bassa di fine del conteggio
    """
    start = PARAM_START*std
    divergenza = UPPER_LIMIT*std
    crash = PARAM_CRASH*std
    return start, divergenza, crash

In [35]:
soglia_start, soglia_divergenza, soglia_crash = calc_treshold(total_stddev)

print(f'{"Soglia Alta":10}\t{"Soglia Bassa":10}\t{"Soglia di Divergenza":10}')
print(f'{soglia_start:>10.4}\t{soglia_divergenza:>10.4}\t{soglia_crash:>10.4}')

Soglia Alta	Soglia Bassa	Soglia di Divergenza
  -0.02042	     20.42	   -0.3063


### Creazione delle Finestre
Si crea un vettore contenente le lunghezze delle finestre.

Si parte da una lunghezza minima definita da `STARTWIN`, fino ad arrivare a `STARTWIN + NUMWINS`.

Si procede a intervalli di giorni dati dal parametro `STEP`.

In [40]:
STEP = 3
windows = np.arange(STARTWIN, STARTWIN+NUMWINS, STEP)
windows

array([ 31,  34,  37,  40,  43,  46,  49,  52,  55,  58,  61,  64,  67,
        70,  73,  76,  79,  82,  85,  88,  91,  94,  97, 100, 103, 106,
       109, 112, 115, 118, 121, 124, 127, 130, 133, 136, 139, 142, 145,
       148, 151, 154, 157, 160, 163, 166, 169, 172, 175, 178])

## Analisi di Stabilità su ogni finestra
Questo pezzo di codice andrà poi implementato in un ciclo su ogni finestra generata in `windows`.

Qui si usa la funzione più importante del codice, `get_stabilvol()`, 
che analizza ogni stock nel DataFrame e crea un nuovo DataFrame in cui raccoglie i **Tempi di Fuga di ogni azione in funzione della Volatilità**.
Per calcolare i tempi di fuga si usa un'altra funzione `count_fht()` che **conta il tempo** tra passaggio dalla Soglia Alta e il passaggio dalla Soglia Bassa dei Ritorni di uno stock. 

Questa fornisce la serie di Volatilità e FHT di _ogni singolo stock_, che viene aggiunta alla serie totale.

Quest'ultimo DataFrame `stabilvol` non distingue più tra gli stocks, è una unica serie di Volatilità e FHT collegato.

In [42]:
def get_stabilvol(database_stocks):
    """
    Raccoglie i Tempi di Fuga FHT in funzione della Volatilità, 
    azione per azione. 
    Parameters
    ----------
    database_stocks : DataFrame
        Ritorni di ogni azione in funzione del tempo
    Returns
    -------
    stabilvol
        DataFrame contenente i FHT in funzione della Volatilità,
        ordinati secondo la Volatilità crescente.
    """
    if VERBOSE: print('\nStarting Analysis of MFHT.')
    # Si inizializza il DataFrame
    stabilvol = pd.DataFrame()
    # Numero di stock analizzati
    n = 0
    # Si fa il ciclo su ogni stock del set di dati
    for stock in database_stocks.columns:
        # Si isola la serie di Ritorni di uno stock
        stock_series = database_stocks[stock]
        # Si passa a indici di riga numerici
        stock_series.reset_index(drop=True, inplace=True)
        # Si calcola la serie di FHT in funzione della Volatilità
        stock_stabilvol = count_fht(stock_series,
                                    soglia_start, soglia_crash, soglia_divergenza,
                                    verbose = False)
        # Si aggiunge questa serie di questa azione alla serie totale
        # di tutte le azioni
        stabilvol = stabilvol.append(stock_stabilvol)
        n = n+1
        # print(f'Analyzed stock {stock}')
    if VERBOSE: print(f'Done.\n{n} stocks analyzed.')
    # Si resetta l'indice di riga del DataFrame (da 0 a n-1)
    stabilvol = stabilvol.reset_index(drop=True)
    # Si ordina il DataFrame per Volatilità crescente
    stabilvol = stabilvol.sort_values('Volatility')
    return stabilvol


### Contare il Tempo di Primo Passaggio


In [45]:
def count_fht(series, high, low, upper, verbose=False):
    """
    Conta quanto tempo passa il Ritorno di una azione a scendere sotto
    la Soglia Bassa dopo essere salito sopra la Soglia Bassa.
    Questo conteggio di giorni sarà il Tempo di Primo Passaggio FHT.
    Calcola quindi in questo intervallo di tempo il valore
    della volatilità del Ritorno dell'azione considerata.
    Parameters
    ----------
    series : Pandas Series
        Serie temporale dei Ritorni di una azione
    high : float
        Soglia Alta sopra la quale salire per avviare il conteggio
    low : float
        Soglia Bassa sotto la quale scendere per fermare il conteggio
    upper : float
        Soglia limite superata la quale annullare il conteggio
    verbose : bool
        Controlla la stampa di output dalla funzione
    Returns
    -------
    stabilvol
        serie di FHT in funzione della Volatilità
    """
    # Inizializza lo stato del conteggio, i Tempi di Fuga e la Volatilità
    count = False
    fht = []
    volatility = []
    # Ciclo su ogni giorno borsistico dell'azione,
    # Si prendono il numero del giorno (timestep)
    # e il valore del Ritorno (level) per quel giorno
    for timestep, level in series.items():
        if verbose: print(f'{level:.6f} - \tH: {high:.6f} - \tL:{low:.6f}')
        # Se non sta già contando e supera la soglia alta: 
        if start_count(level, high, upper, count):
            # comincia il conteggio
            count = True
            # e salva il primo giorno del conteggio
            start = timestep
            if verbose:
                print(f'{level:.8f} > {high:.8f}')
                print('Count initiated')
        # Se sta contando e supera la soglia bassa:
        if stop_count(level, low, count):
            # ferma il conteggio
            count = False
            # e salva l'ultimo giorno di conteggio
            end = timestep
            if verbose:
                print(f'{level:.8f} < {low:.8f}')
                print(f'Count stop after {end-start} days')
            # Calcola il tempo in cui è stato attivo il conteggio
            counting_time = end-start
            # Se il conteggio è regolare, né troppo piccolo né troppo grande:
            if TAU_MIN <= counting_time <= TAU_MAX:
                # aggiunge il conteggio ai Tempi di Fuga
                fht.append(counting_time)
                # calcola la Volatilità ristretta ai giorni del conteggio
                local_volatility = series[start:end].std()
                # e la aggiunge alle Volatilità
                volatility.append(local_volatility)
    # Si crea un dictionary dagli array di Volatilità e FHT
    stabilvol = {'Volatility': volatility, 'FHT': fht}
    # che si trasforma in un DataFrame 
    # (con colonne "Volatilità" e "FHT")
    stabilvol = pd.DataFrame(stabilvol)
    # stabilvol = pd.Series(fht, index=volatility)
    return stabilvol

Per controllare il conteggio si usano due funzioni ausiliarie:

In [46]:
def start_count(level, high, upper, count):
    """
    Indica se iniziare il conteggio o meno.
    Inizia il conteggio se il livello del Ritorno è
    maggiore della Soglia Alta e minore della Soglia di Divergenza.
    Inoltre, il conteggio non deve già essere avviato.
    Parameters
    ----------
    level : float
        Valore del Ritorno di una azione
    high : float
        Soglia Alta da superare per avviare il conteggio
    upper : float
        Soglia di Divergenza da *non* superare per avere 
        un conteggio valido
    count : bool
        Stato del conteggio: già avviato oppure no
    Returns
    -------
    start
        Variabile booleana per avviare o meno il conteggio
    """
    if level>=high and level < upper and not count:
        start = True
    else:
        start = False
    return start

def stop_count(level, low, count):
    """
    Indica se fermare il conteggio o meno.
    Ferma il conteggio se questo è già avviato e se
    il livello del Ritorno è minore della Soglia Bassa.
    Parameters
    ----------
    level : float
        Valore del Ritorno di una azione
    low : float
        Soglia Bassa da superare per fermare il conteggio
    count : bool
        Stato del conteggio: già avviato oppure no
    Returns
    -------
    stop
        Variabile booleana per fermare o meno il conteggio
    """
    if level<=low and count:
        stop = True
    else:
        stop = False
    return stop

In [50]:
stabilvol = get_stabilvol(database_stocks.iloc[-100:-1])
stabilvol

,Volatility,FHT
184,0.000000,6.0
984,0.000000,2.0
676,0.003416,2.0
193,0.008544,2.0
935,0.011172,2.0
...,...,...
1362,6.476880,21.0
1356,7.347900,9.0
903,7.445510,13.0
904,9.900867,17.0


### Pulizia dei risultati
I valori di Volatilità e FHT ottenuti si possono sistemare per essere visualizzati meglio.

È importante in particolare raggruppare i tempi in **canali discreti** di Volatilità per mostrare i **Tempi Medi di Primo Passaggio** in questi canali. 

Questo raggruppamento è effettuato dalla funzione `make_binned()`

In [55]:
def make_binned(df, nbins=100):
    """
    Crea degli intervalli in cui divide la prima colonna
    del DataFrame e raggruppa in questi i valori del DataFrame
    secondo valor medio.
    Parameters
    ----------
    df : DataFrame
        Dati da raggruppare secondo intervalli generati
        dividendo la prima colonna
    nbins : int
        Numero di intervalli equispaziati
        in cui dividere la prima colonna
    Returns
    -------
    df 
        DataFrame con valori raggruppati negli intervalli
    """
    # Crea gli Intervalli a partire dalla Volatilità
    bins = pd.cut(df.iloc[:,0], nbins)
    # Inserisce gli Intervalli nel DataFrame
    df.insert(0, 'Bins', bins)
    # Raggruppa le colonne del DataFrame per valor medio
    # in ogni intervallo
    df = df.groupby(['Bins']).mean()
    return df

In [69]:
# Filtra via la coda lunga di Volatilità
stabilvol_filtered = stabilvol.loc[stabilvol['Volatility'] <= 1.0]
# E raggruppa FHT in canali di Volatilità secondo il valor medio nell'intervallo
stabilvol_filtered = make_binned(stabilvol_filtered, nbins=NUMBINS)

In [70]:
# Stampa il DataFrame filtrato e mediato, con gli Intervalli
print(stabilvol_filtered.head())
print('\n. . .\n')
print(stabilvol_filtered.tail())

                Volatility        FHT
Bins                                 
(-0.001, 0.02]    0.007608   2.571429
(0.02, 0.04]      0.030433   7.000000
(0.04, 0.06]      0.049963   8.000000
(0.06, 0.08]      0.075431  18.352941
(0.08, 0.1]       0.089970  22.600000

. . .

              Volatility        FHT
Bins                               
(0.9, 0.92]     0.910076  18.500000
(0.92, 0.94]    0.931990   9.333333
(0.94, 0.96]    0.950746  15.000000
(0.96, 0.98]    0.971022  16.250000
(0.98, 1.0]     0.988146  18.000000
